In [7]:
import sys
import os
from pathlib import Path
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import importlib

sys.path.append(os.path.abspath(".."))

import src.models.mlp.mlp_cv_trainer as cv
import src.utils.optuna_visualizer as opv
import src.utils.telegram as te

In [8]:
# Load data
env_path = Path.cwd().parent / ".env"
load_dotenv(dotenv_path=env_path)
url = os.environ.get("OPTUNA_STORAGE_URL")

l1_tr_df2 = pd.read_parquet("../artifacts/features/l1/l1_tr_df2.parquet")
l1_test_df2 = pd.read_parquet("../artifacts/features/l1/l1_test_df2.parquet")
l1_tr_df4 = pd.read_parquet("../artifacts/features/l1/l1_tr_df4.parquet")
l1_test_df4 = pd.read_parquet("../artifacts/features/l1/l1_test_df4.parquet")

In [ ]:
# Check tuning results
study_name = "l1_mlp_v2"

study = opv.OptunaVisualizer(study_name, url)
study.visualize_optimization()
study.print_trials_table()

In [ ]:
# Create OOF and test predictions
importlib.reload(cv)
params = {
    "hidden_dim1": 832,
    "hidden_dim2": 672,
    "hidden_dim3": 352,
    "hidden_dim4": 288,
    "batch_size": 864,
    "lr": 0.0032694640869831816,
    "eta_min": 0.0002853657880820916,
    "dropout_rate": 0.30000000000000004,
    "activation": "ReLU"
}
trainer = cv.MLPCVTrainer(**params)

oof, test_preds = trainer.fit(l1_tr_df2, l1_test_df2)
np.save("../artifacts/preds/l1/oof_single_16.npy", oof)
np.save("../artifacts/preds/l1/test_single_16.npy", test_preds)

In [15]:
te.send_telegram_message("MLP Training complete!")

✅ Message sent successfully.
